In [1]:
import os
import pickle
import torch
import numpy as np
import IPython.display as ipd

In [2]:
from math import ceil
from librosa.filters import mel
from librosa.util import normalize
from melgan.interface import *

In [3]:
from factory.autovc_pytorch_org import AutoVC_ORG
from factory.autovc_pytorch_org_dc import AutoVC_ORG_DC
from factory.autovc_pytorch_metaformer_conv_v2 import AutoVC_MetaFormer_CONV_V2
from factory.autovc_pytorch_metaformer_pool_v2 import AutoVC_MetaFormer_Pool_V2
#from factory.autovc_pytorch_mlp_mixer import AutoVC_MLP_MIXER
#from factory.autovc_pytorch_metaformer_pool_v3 import AutoVC_MetaFormer_Pool_V3

In [4]:
LEN_CROP = 176
DIM_NECK = 44
DIM_EMB = 256
D_MODEL = 80
FREQ = 22
BATCH_SIZE = 2
DEVICE = "cpu"

In [5]:
org = AutoVC_ORG(DIM_NECK,DIM_EMB,2*DIM_EMB,FREQ)
org.load_state_dict(torch.load("model/autovc_pytorch_org_tw.pt", map_location=DEVICE))
orgdc = AutoVC_ORG_DC(DIM_NECK,DIM_EMB,2*DIM_EMB,FREQ)
orgdc.load_state_dict(torch.load("model/autovc_pytorch_org_dc_tw.pt", map_location=DEVICE))
conv2 = AutoVC_MetaFormer_CONV_V2(DIM_NECK,DIM_EMB,2*DIM_EMB,FREQ)
conv2.load_state_dict(torch.load("model/autovc_pytorch_meta_conv_tw.pt", map_location=DEVICE))
pool = AutoVC_MetaFormer_Pool_V2(DIM_NECK,DIM_EMB,2*DIM_EMB,FREQ)   
pool.load_state_dict(torch.load("model/autovc_pytorch_meta_pool_tw.pt", map_location=DEVICE))

#mlpmixer = AutoVC_MLP_MIXER(DIM_NECK,DIM_EMB,2*DIM_EMB,FREQ)
#mlpmixer.load_state_dict(torch.load("model/autovc_pytorch_mlp_mixer_1000000_11_11.pt", map_location=DEVICE))

#metaformerv3 = AutoVC_MetaFormer_Pool_V3(DIM_NECK,DIM_EMB,2*DIM_EMB,FREQ)   
#metaformerv3.load_state_dict(torch.load("model/autovc_pytorch_metatransformer_pool_v3_12_12_zh.pt", map_location=DEVICE))

<All keys matched successfully>

In [6]:
vocoder = MelVocoder(model_name='model/static/multi_speaker')

In [7]:
metadata = pickle.load(open('train_data/train.pkl', "rb"))

In [277]:
# high qua, 6,10,11,12,13
# 29,12,13 -> 35

# 0~5~15

# (35 柯 P)
source = 0
target = 5
sound = 15

In [278]:
for i in range(42): 
    if int(metadata[i][0]) == 40 or int(metadata[i][0]) == 41:
        print(i)

35
36


In [279]:
metadata[source][0]
emb_org = np.expand_dims(metadata[source][1],axis=0)
emb_org.shape

(1, 256)

In [280]:
uttr_org = np.load(f"train_data/{metadata[source][sound]}")[0:176]
uttr_org  = np.expand_dims(uttr_org ,axis=0)
uttr_org.shape

(1, 176, 80)

In [281]:
metadata[target][0]
emb_trg =  np.expand_dims(metadata[target][1],axis=0)
emb_trg.shape

(1, 256)

## 原始聲音

In [282]:
audio_s = vocoder.inverse(torch.from_numpy(np.transpose(uttr_org,(0,2,1))))
ipd.Audio(audio_s,rate = 22050)

## 目標聲音

In [283]:
uttr_target = np.expand_dims(np.load(f"train_data//{metadata[target][sound]}")[50:226].T,axis=0)
audio_s = vocoder.inverse(torch.from_numpy(uttr_target))
ipd.Audio(audio_s,rate = 22050)

In [284]:
def pad_along_axis(array: np.ndarray, target_length: int, axis: int = 0):
    pad_size = target_length - array.shape[axis]
    if pad_size <= 0:
        return array
    npad = [(0, 0)] * array.ndim
    npad[axis] = (0, pad_size)
    return np.pad(array, pad_width=npad, mode='constant', constant_values=0)

In [285]:
_,res_org,_ = org(torch.from_numpy(uttr_org).to(DEVICE),torch.from_numpy(emb_org).to(DEVICE),torch.from_numpy(emb_trg).to(DEVICE))
_,res_orgdc,_ = orgdc(torch.from_numpy(uttr_org).to(DEVICE),torch.from_numpy(emb_org).to(DEVICE),torch.from_numpy(emb_trg).to(DEVICE))
_,res_conv2,_ = conv2(torch.from_numpy(uttr_org).to(DEVICE),torch.from_numpy(emb_org).to(DEVICE),torch.from_numpy(emb_trg).to(DEVICE))
_,res_pool,_ = pool(torch.from_numpy(uttr_org).to(DEVICE),torch.from_numpy(emb_org).to(DEVICE),torch.from_numpy(emb_trg).to(DEVICE))
#_,res_mlp,_ = mlpmixer(torch.from_numpy(uttr_org).to(DEVICE),torch.from_numpy(emb_org).to(DEVICE),torch.from_numpy(emb_trg).to(DEVICE))
#_,res_metaformer_v2,_ = metaformerv2(torch.from_numpy(uttr_org).to(DEVICE),torch.from_numpy(emb_org).to(DEVICE),torch.from_numpy(emb_trg).to(DEVICE))
#_,res_metaformer_v3,_ = metaformerv3(torch.from_numpy(uttr_org).to(DEVICE),torch.from_numpy(emb_org).to(DEVICE),torch.from_numpy(emb_trg).to(DEVICE))

In [286]:
org_trg = res_org[0,  :, :].transpose(1,2)
orgdc_trg = res_orgdc[0,  :, :].transpose(1,2)
conv2_trg = res_conv2[0,  :, :].transpose(1,2)
pool_trg = res_pool[0,  :, :].transpose(1,2)
#mlp_trg = res_mlp[0,  :, :].transpose(1,2)
#metaformer_v2_trg = res_metaformer_v2[0,  :, :].transpose(1,2)
#metaformer_v3_trg = res_metaformer_v3[0,  :, :].transpose(1,2)

In [287]:
org_audio = vocoder.inverse(org_trg)
orgdc_audio = vocoder.inverse(orgdc_trg)
conv2_audio = vocoder.inverse(conv2_trg)
pool_audio = vocoder.inverse(pool_trg)
#mlp_audio = vocoder.inverse(mlp_trg)
#metaformer_v2_audio = vocoder.inverse(metaformer_v2_trg)
#metaformer_v3_audio = vocoder.inverse(metaformer_v3_trg)

## 轉換聲音

In [288]:
ipd.Audio(org_audio,rate = 22000)

In [289]:
ipd.Audio(orgdc_audio,rate = 22000)

In [290]:
ipd.Audio(conv2_audio,rate = 22000)

In [291]:
ipd.Audio(pool_audio,rate = 22050)

In [181]:
#ipd.Audio(mlp_audio,rate = 22050)

In [684]:
#ipd.Audio(metaformer_v3_audio,rate = 22050)

In [2224]:
from scipy.io.wavfile import write
def save_to_result(audio,filename, rate=22050):
    write(filename, rate,audio)

In [2226]:
save_to_result(org_audio.squeeze().numpy(),f'test.wav')

In [2302]:
"""
import os
import pickle
import numpy as np
import librosa
from scipy import signal
from scipy.signal import get_window
from librosa.filters import mel
from librosa.util import normalize
from numpy.random import RandomState
def mel_gan_handler(x, fft_length = 1024, hop_length = 256,sr = 22000):
    wav = normalize(x)
    p = (fft_length - hop_length) // 2
    wav = np.squeeze(np.pad(wav, (p, p), "reflect"))
    fft = librosa.stft(
                       wav, 
                       n_fft = fft_length, 
                       hop_length = hop_length,
                       window = 'hann',
                       center = False
                     )
    # 這裡的 abs 是 sqrt(實部**2 + 虛部**2)
    mag = abs(fft)
    mel_basis = mel(sr, 1024, fmin = 0.0 , fmax=None, n_mels=80)
    mel_output = np.dot(mel_basis,mag)
    log_mel_spec = np.log10(np.maximum(1e-5,mel_output)).astype(np.float32)
    return log_mel_spec
target_fs = 22000
# Read audio file
x, fs = librosa.load("test_data/test1/22.mp3")
# resample to target_rate
x = librosa.resample(x, fs, target_fs)
S = mel_gan_handler(x).T 
S.shape
tmp = pad_along_axis(S,176)
melsp = tmp[np.newaxis,:, :]
uttr_org = melsp[::,0:176]
print(uttr_org.shape)
"""

(1, 176, 80)
